In [ ]:
pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 60.8 MB/s eta 0:00:00


In [ ]:
import fitz  # PyMuPDF
import os

def extract_pages_as_png(pdf_path, output_folder):
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    pdf_document = fitz.open(pdf_path)

    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)

        zoom = 2  # Zoom factor for higher resolution (e.g., 2x)
        mat = fitz.Matrix(zoom, zoom)

        pix = page.get_pixmap(matrix=mat, alpha=False)

        output_path = os.path.join(output_folder, f"page_{page_num + 1}.png")
        pix.save(output_path)
        print(f"Saved {output_path}")

    print(f"All pages extracted and saved to {output_folder}")

if __name__ == "__main__":
    pdf_path = "/content/test1.pdf"  # Replace with your PDF file path
    output_folder = "output_images"  # Replace with your desired output folder
    extract_pages_as_png(pdf_path, output_folder)


Saved output_images/page_1.png
All pages extracted and saved to output_images


In [ ]:
pip install pillow


In [ ]:
import re
from PIL import Image
import requests
from io import BytesIO

from transformers import DonutProcessor, VisionEncoderDecoderModel
#from datasets import load_dataset
import torch

processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base-finetuned-cord-v2")
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base-finetuned-cord-v2")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


# load document image
image_path = "/content/output_images/page_1.png"
image = Image.open(image_path)

# prepare decoder inputs
task_prompt = "<s_cord-v2>"
decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids

pixel_values = processor(image, return_tensors="pt").pixel_values

outputs = model.generate(
    pixel_values.to(device),
    decoder_input_ids=decoder_input_ids.to(device),
    max_length=model.decoder.config.max_position_embeddings,
    pad_token_id=processor.tokenizer.pad_token_id,
    eos_token_id=processor.tokenizer.eos_token_id,
    use_cache=True,
    bad_words_ids=[[processor.tokenizer.unk_token_id]],
    return_dict_in_generate=True,
)

sequence = processor.batch_decode(outputs.sequences)[0]
sequence = sequence.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
sequence = re.sub(r"<.*?>", "", sequence, count=1).strip()  # remove first task start token
print(processor.token2json(sequence))

Config of the encoder: <class 'transformers.models.donut.modeling_donut_swin.DonutSwinModel'> is overwritten by shared encoder config: DonutSwinConfig {
  "attention_probs_dropout_prob": 0.0,
  "depths": [
    2,
    2,
    14,
    2
  ],
  "drop_path_rate": 0.1,
  "embed_dim": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "image_size": [
    1280,
    960
  ],
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-05,
  "mlp_ratio": 4.0,
  "model_type": "donut-swin",
  "num_channels": 3,
  "num_heads": [
    4,
    8,
    16,
    32
  ],
  "num_layers": 4,
  "patch_size": 4,
  "path_norm": true,
  "qkv_bias": true,
  "transformers_version": "4.47.1",
  "use_absolute_embeddings": false,
  "window_size": 10
}

Config of the decoder: <class 'transformers.models.mbart.modeling_mbart.MBartForCausalLM'> is overwritten by shared decoder config: MBartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "add_

{'nm': 'Shubham Jaiswald suisai.536.in'}


In [ ]:
pip install langchain


In [ ]:
pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [ ]:
%pip install --upgrade --quiet pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 10.7 MB/s eta 0:00:00


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = (
    "/content/test1.pdf"
)
loader = PyPDFLoader(file_path)
pages = loader.load_and_split()

pages[0]

Document(metadata={'source': '/content/test1.pdf', 'page': 0}, page_content='EDCATION \nB. Tech. \nShubham Jaiswal \nDegree/Certiticate \nEXPERIENCE \nRoll No.: 2101060G6 \nSenior Secoudarv Svcondary \nB.Tech - Bioscience and Bioengineering Indian Instit ute (OF Techuology, Guwaluti \n" Transled Medtech \n" CaupusWay \nWeo Derelopment Intern \nPersonal Project \nPersonal Project \nGroup Project \nIudian Iustitute of Technology, Guwabuti CBSE Board CBSE Board \nGroup Project, Coding Club, lITG \n-Deveoped responsive Ul using React, CSS, and JavaScript, cnsuring dynamic rendering and iuteractive featurcs. - lmpleunented UI compouents like navigation bars And modals using Framer Motion and Material-UI Opimized front-end perforn1ance by minifying JavaScript and CSS, implcmenting lazy loading with React.lazy(). PROJECTS \nIustitute/Board \nCodeforces, MaX-rating -1306 (Pupil) -Codeforces Roumd 169: Sccured Global Rank 685 among 20k+ participants. . Hacker Rank, Reached 6 star, in problem-so